In [3]:
import numpy as np
import pandas as pd
import os
import math
import re
import jieba
import datetime
from multiprocessing import Pool
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [4]:
from sklearn import metrics

In [5]:
def label_day(indexs, gap, percentage, listed_stock_2018):
    MediaTek = pd.DataFrame(columns = ['date','class'])
    print("all_days ", len(indexs))
    
    for i in range(len(indexs) - gap):
        j = i + gap
        if float(listed_stock_2018.iloc[indexs[i]]['收盤價(元)'])* (1 + percentage) < float(listed_stock_2018.iloc[indexs[j]]['收盤價(元)']):
            MediaTek = MediaTek.append([{'date':listed_stock_2018.iloc[indexs[i]]['年月日'],'class': 1}])
        elif float(listed_stock_2018.iloc[indexs[i]]['收盤價(元)'])* (1-percentage) > float(listed_stock_2018.iloc[indexs[j]]['收盤價(元)']):
            MediaTek = MediaTek.append([{'date':listed_stock_2018.iloc[indexs[i]]['年月日'],'class': -1}])
        else:
            MediaTek = MediaTek.append([{'date':listed_stock_2018.iloc[indexs[i]]['年月日'],'class': 0}])
    return MediaTek

In [6]:
def label_article(news, deal,MediaTek):
    label_news_2018 = pd.DataFrame(columns = ['id','post_time','content','class'])

    for index, val in news.iterrows():
        if index in deal:
            for j in range(len(MediaTek["date"])):
                if val["post_time"] == MediaTek["date"].values[j]:
                    if MediaTek["class"].values[j] == 1:
                        label_news_2018 = label_news_2018.append([{'id':val['id'], 'post_time':val['post_time'],'content':val['content'], 'class':1}])
                    elif MediaTek["class"].values[j] == -1:
                        label_news_2018 = label_news_2018.append([{'id':val['id'], 'post_time':val['post_time'],'content':val['content'], 'class':0}])
                    break

    return label_news_2018

In [7]:
def token(ver):
    print(ver['class'].value_counts())
    
    ver_up = ver[ver['class'] == 1]
    ver_down = ver[ver['class'] == 0]
    
    text_all = []
    label_all = []
    text_up = ''
    sent_up = []

    for i in ver_up['content']:
        text_up = i
        text_up = re.sub(r'[^\w]','',text_up)
        text_up = re.sub(r'[A-Za-z0-9]','',text_up)
        temp_up = ''
        segments_up=[]
        remainderWords_up=[]
        segments_up = jieba.cut(text_up, cut_all=False)
        remainderWords_up = list(filter(lambda a: a not in stopword and a != '\n', segments_up))
        cnt = 0
        for k in remainderWords_up:
            if (cnt == 0) or (cnt == len(remainderWords_up)):
                temp_up += (k)
            else:
                temp_up += ' '
                temp_up += k
            cnt += 1
        sent_up.append(temp_up)
    print("up: ", len(sent_up))


    text_all = []
    label_all = []
    text_down = ''
    sent_down = []
    
    for i in ver_down['content']:
        text_down = i
        text_down = re.sub(r'[^\w]','',text_down)
        text_down = re.sub(r'[A-Za-z0-9]','',text_down)
        temp_down = ''
        segments_down=[]
        remainderWords_down=[]
        segments_down = jieba.cut(text_down, cut_all=False)
        remainderWords_down = list(filter(lambda a: a not in stopword and a != '\n', segments_down))
        cnt = 0
        for k in remainderWords_down:
            if (cnt == 0) or (cnt == len(remainderWords_down)):
                temp_down += (k)
            else:
                temp_down += ' '
                temp_down += k
            cnt += 1
        sent_down.append(temp_down)
    print("down: ",len(sent_down))

    text_all = []
    label_all = []
    text_down = ''
    sent = []

    for i in ver['content']:
        text_up = i
        text_up = re.sub(r'[^\w]','',text_up)
        text_up = re.sub(r'[A-Za-z0-9]','',text_up)
        temp_up = ''
        segments_up=[]
        remainderWords_up=[]
        segments_up = jieba.cut(text_up, cut_all=False)
        remainderWords_up = list(filter(lambda a: a not in stopword and a != '\n', segments_up))
        cnt = 0
        for k in remainderWords_up:
            if (cnt == 0) or (cnt == len(remainderWords_up)):
                temp_up += (k)
            else:
                temp_up += ' '
                temp_up += k
            cnt += 1
        sent.append(temp_up)
    print("all: ",len(sent))
    
    return sent_up, sent_down, sent

In [8]:
def find_keyword(a, b, c, sent_up ,sent_down):
    cv_up = CountVectorizer(min_df = a ,max_df = b ,max_features =c)
    cv_fit_up =cv_up.fit_transform(sent_up)
    
    print("up_keyword: ",cv_fit_up.toarray().shape)
    key_up = cv_up.get_feature_names()
    
    cv_down = CountVectorizer(min_df = a ,max_df = b ,max_features =c)
    cv_fit_down =cv_down.fit_transform(sent_down)
    
    print("down_keyword: ",cv_fit_down.toarray().shape)
    
    key_down = cv_down.get_feature_names()
    
    key = key_up
    for i in key_down:
        if i not in key_up:
            key.append(i)
    return key_up, key_down, key 

In [9]:
stopword = []
with open ('stopword.txt','r',encoding = 'utf-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopword.append(data)

In [10]:
news = pd.read_csv("news.csv",encoding = "big5")
n = len(news)
news.dropna(inplace = True)

In [11]:
for i in range(len(news)):
    news.iloc[i]['post_time'] = news.iloc[i]['post_time'].split()[0]

In [12]:
listed_stock_2018 = pd.read_csv("listed_stock_2018.csv")
listed_stock_2017 = pd.read_csv("listed_stock_2017.csv")
listed_stock_2016 = pd.read_csv("listed_stock_2016.csv")
listed_stock_2018 = listed_stock_2018.append(listed_stock_2017,ignore_index = True)
listed_stock_2018 = listed_stock_2018.append(listed_stock_2016,ignore_index = True)

In [13]:
find = listed_stock_2018['證券代碼'] == "2454 聯發科"
indexs = list(find[find.values == True].index)
indexs.reverse()
print("all days: ", len(indexs))

# for i in indexs:
#     print(listed_stock_2018.iloc[i]['年月日'])

all days:  737


In [14]:
stock = "聯發科"
deal = [] # remember the articles index will be dealed with
#article_list = [article(index) for index in range(n+1)] # extra new for index = 0 

for index, val in news.iterrows(): # for 90000
    if stock in val["content"] + val["title"]:
        deal.append(index)

In [15]:
print("all articles: ", len(deal))
print(deal[:10])

all articles:  5185
[69, 147, 238, 290, 307, 371, 441, 485, 486, 517]


In [35]:
MediaTek = label_day(indexs, 1, 0.03,  listed_stock_2018)
label_news_2018 = label_article(news, deal, MediaTek)  
all_article = token(label_news_2018) # sent_up,sent_down,sent
key = find_keyword(0.01, 0.6, 2000, all_article[0], all_article[1]) # key_up, key_down, key

all_days  737
 0    655
-1     41
 1     40
Name: class, dtype: int64
736
0    283
1    262
Name: class, dtype: int64
up:  262
down:  283
all:  545
up_keyword:  (262, 2000)
down_keyword:  (283, 2000)


In [36]:
# all
cv = CountVectorizer(vocabulary = key[2])
cv_fit =cv.fit_transform(all_article[2])
all = len(cv_fit.toarray())
print(cv_fit.toarray().shape)

X_data = cv_fit.toarray()
Y_data = label_news_2018['class'].to_numpy(dtype=int)

valid = int(math.floor(all * 0.8)) # Round down
print('valid :', valid)

X_train = X_data[:valid]
X_test = X_data[valid:]

Y_train = Y_data[:valid]
Y_test = Y_data[valid:]

(545, 2602)
valid : 436


In [37]:
K = [1,2,3,4,5,10,15,20,25,30,35,40,45,50]
for k in K:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,Y_train)
    pred = knn.predict(X_test)
    print(k)
    print(confusion_matrix(Y_test,pred))
    
    print('R=',metrics.recall_score(Y_test, pred))
    print('P=',metrics.precision_score(Y_test, pred))
    print('Accuracy=',metrics.accuracy_score(Y_test, pred))
    print('F1=',metrics.f1_score(Y_test, pred))

1
[[39 27]
 [32 11]]
R= 0.2558139534883721
P= 0.2894736842105263
Accuracy= 0.45871559633027525
F1= 0.2716049382716049
2
[[57  9]
 [38  5]]
R= 0.11627906976744186
P= 0.35714285714285715
Accuracy= 0.5688073394495413
F1= 0.1754385964912281
3
[[52 14]
 [33 10]]
R= 0.23255813953488372
P= 0.4166666666666667
Accuracy= 0.5688073394495413
F1= 0.29850746268656714
4
[[60  6]
 [38  5]]
R= 0.11627906976744186
P= 0.45454545454545453
Accuracy= 0.5963302752293578
F1= 0.18518518518518515
5
[[56 10]
 [36  7]]
R= 0.16279069767441862
P= 0.4117647058823529
Accuracy= 0.5779816513761468
F1= 0.23333333333333336
10
[[62  4]
 [38  5]]
R= 0.11627906976744186
P= 0.5555555555555556
Accuracy= 0.6146788990825688
F1= 0.19230769230769232
15
[[44 22]
 [30 13]]
R= 0.3023255813953488
P= 0.37142857142857144
Accuracy= 0.5229357798165137
F1= 0.3333333333333333
20
[[35 31]
 [25 18]]
R= 0.4186046511627907
P= 0.3673469387755102
Accuracy= 0.48623853211009177
F1= 0.391304347826087
25
[[25 41]
 [19 24]]
R= 0.5581395348837209
P= 0